<img src="tic3151.png" style="height: 70px; width: 200px" align=left> 
<img src="logo-MMU.png" style="height: 70px; width: 200px" align=right> 

# Lab 4: BFS and A* Search

### Learning Objectives:

Towards the end of the lab session, you will be able to:

- define and code Breadth-First Search
- define and code A* Search 

# Breadth-First Search

In [1]:
def BFS(m):
    #initialization    
    start=(m.rows,m.cols)
    explored=[start]
    frontier=[start]
    
    print("original frontier=",frontier)
    print("original explored=",explored)
    
    bfsPath={}
    searchPath=[]
    # continue loop as long as frontier !=0
    while len(frontier)>0:
        # remember to pop based on FIFO
        # your answer here
        currCell = frontier.pop(0)
        searchPath.append(currCell)
       
        
        # if current cell == goal then end
        if currCell ==(1,1):
            break
        
        # this loop is to look into all 4 directions.
        # your answer here...##########
        for d in "ESNW":
            if m.maze_map[currCell][d]==True:
                if d=="E":
                    childCell=(currCell[0], currCell[1]+1)
                elif d=="W":
                    childCell=(currCell[0], currCell[1]-1)
                elif d=="S":
                    childCell=(currCell[0]+1, currCell[1])
                elif d=="N":
                    childCell=(currCell[0]-1, currCell[1])
                
                if childCell in explored:
                    continue
                
                explored.append(childCell)
                frontier.append(childCell)
                
                print("frontier=",frontier)
                print("explored=",explored)
                
                bfsPath[childCell]=currCell
    
    print("=============================================")
    # print out content of bfsPath    
    print("{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in bfsPath.items()) + "}")
     
    fwdPath={}
    cell=(1,1)
    
    print("=============================================")
    
    while cell != start:  
        fwdPath[bfsPath[cell]] = cell  
        cell = bfsPath[cell] 
    
    print("=============================================")
    print("{" + "\n".join("{!r}: {!r},".format(k, v) for k, v in fwdPath.items()) + "}")
    
    return fwdPath

In [2]:
from pyamaze import maze, agent

m = maze(4,4)
m.CreateMaze(loadMaze="maze.csv")
path=BFS(m)
a=agent(m,footprints=True)
m.tracePath({a:path})
m.run()

original frontier= [(4, 4)]
original explored= [(4, 4)]
frontier= [(3, 4)]
explored= [(4, 4), (3, 4)]
frontier= [(2, 4)]
explored= [(4, 4), (3, 4), (2, 4)]
frontier= [(2, 4), (3, 3)]
explored= [(4, 4), (3, 4), (2, 4), (3, 3)]
frontier= [(3, 3), (1, 4)]
explored= [(4, 4), (3, 4), (2, 4), (3, 3), (1, 4)]
frontier= [(1, 4), (4, 3)]
explored= [(4, 4), (3, 4), (2, 4), (3, 3), (1, 4), (4, 3)]
frontier= [(4, 3), (1, 3)]
explored= [(4, 4), (3, 4), (2, 4), (3, 3), (1, 4), (4, 3), (1, 3)]
frontier= [(1, 3), (4, 2)]
explored= [(4, 4), (3, 4), (2, 4), (3, 3), (1, 4), (4, 3), (1, 3), (4, 2)]
frontier= [(4, 2), (2, 3)]
explored= [(4, 4), (3, 4), (2, 4), (3, 3), (1, 4), (4, 3), (1, 3), (4, 2), (2, 3)]
frontier= [(4, 2), (2, 3), (1, 2)]
explored= [(4, 4), (3, 4), (2, 4), (3, 3), (1, 4), (4, 3), (1, 3), (4, 2), (2, 3), (1, 2)]
frontier= [(2, 3), (1, 2), (4, 1)]
explored= [(4, 4), (3, 4), (2, 4), (3, 3), (1, 4), (4, 3), (1, 3), (4, 2), (2, 3), (1, 2), (4, 1)]
frontier= [(3, 1)]
explored= [(4, 4), (3, 4)

# Exercise

(1) Explain the output of dfsPath and fwdpath dictionary. 

(2) What should be done so that the DFS is using this order: E -> W -> N -> S. That is to explore "E" first, then "W", "N", and "S".


# Maze and A* Search

In [1]:
from pyamaze import maze,agent,COLOR,textLabel
from queue import PriorityQueue

In [2]:
# Declaring the heuristic function as manhattan distance

def h(cell1,cell2):
    x1,y1=cell1
    x2,y2=cell2
    return abs(x1-x2) + abs(y1-y2)

In [3]:
def aStar(m):
    start=(m.rows,m.cols)
    g_score={cell:float('inf') for cell in m.grid}
    g_score[start]=0
    f_score={cell:float('inf') for cell in m.grid}
    f_score[start]=h(start,(1,1))

    open=PriorityQueue()
    open.put((h(start,(1,1)),h(start,(1,1)),start))
    aPath={}
    searchPath=[]
    
    # your answer here...
    while not open.empty():
        currCell=open.get()[2]
        searchPath.append(currCell)
      
        # if current cell == goal then end
        if currCell ==(1,1):
            break
            
        for d in "ESNW":
            if m.maze_map[currCell][d]==True:
                if d=="E":
                    childCell=(currCell[0], currCell[1]+1)
                if d=="W":
                    childCell=(currCell[0], currCell[1]-1)
                if d=="S":
                    childCell=(currCell[0]+1, currCell[1])
                if d=="N":
                    childCell=(currCell[0]-1, currCell[1])
                    
                temp_g_score=g_score[currCell]+1
                temp_f_score=temp_g_score=h(childCell,(1,1))
                
                if temp_f_score < f_score[childCell]:
                    g_score[childCell]=temp_g_score
                    f_score[childCell]=temp_f_score
                    open.put((temp_f_score, h(childCell, (1,1)), childCell))
                    aPath[childCell]=currCell
                
    fwdPath={}
    cell=(1,1)
    
    while cell!=start:
        fwdPath[aPath[cell]]=cell
        cell=aPath[cell]
    return fwdPath,aPath, searchPath

### Visualize the aStar path

In [4]:
from pyamaze import maze, agent, COLOR

m = maze()
m.CreateMaze(loadMaze='aStar-Maze.csv')

path=aStar(m) 
a=agent(m,footprints=True)

# this is to demo the search path
b=agent(m,footprints=True,color=COLOR.yellow,filled=True)
m.tracePath({b:path[2]},showMarked=True)
print(path[2])

# this is to demo the fwdPath
m.tracePath({a:path[0]}) 
l=textLabel(m, "A star Path Length", len(path[0])+1)
print(path[0])

m.run()

[(9, 10), (8, 10), (7, 10), (8, 9), (8, 8), (8, 7), (7, 7), (7, 6), (9, 9), (9, 8), (9, 7), (9, 6), (8, 6), (8, 5), (8, 4), (7, 4), (7, 5), (6, 5), (6, 6), (6, 7), (6, 8), (7, 8), (7, 9), (6, 9), (6, 10), (5, 10), (4, 10), (3, 10), (2, 10), (1, 10), (1, 9), (2, 9), (3, 9), (3, 8), (3, 7), (2, 7), (1, 7), (1, 6), (1, 5), (1, 4), (1, 3), (1, 8), (2, 8), (5, 9), (4, 9), (4, 8), (4, 7), (4, 6), (3, 6), (2, 6), (2, 5), (3, 5), (3, 4), (2, 4), (2, 3), (2, 2), (1, 2), (1, 1)]
{(1, 2): (1, 1), (2, 2): (1, 2), (2, 3): (2, 2), (2, 4): (2, 3), (3, 4): (2, 4), (3, 5): (3, 4), (2, 5): (3, 5), (2, 6): (2, 5), (3, 6): (2, 6), (4, 6): (3, 6), (4, 7): (4, 6), (4, 8): (4, 7), (4, 9): (4, 8), (5, 9): (4, 9), (5, 10): (5, 9), (6, 10): (5, 10), (6, 9): (6, 10), (7, 9): (6, 9), (7, 8): (7, 9), (6, 8): (7, 8), (6, 7): (6, 8), (6, 6): (6, 7), (6, 5): (6, 6), (7, 5): (6, 5), (7, 4): (7, 5), (8, 4): (7, 4), (8, 5): (8, 4), (8, 6): (8, 5), (9, 6): (8, 6), (9, 7): (9, 6), (9, 8): (9, 7), (9, 9): (9, 8), (9, 10): 

# Exercise

(1) Include the code to display the content of the solution path and calculate the solution path cost.
